# SIGIR17 Attractions: Baseline ETL Job
<img src="https://i.imgur.com/ifo76q1.png" width="1000"><br>
##### Purpose
>This notebook will demonstrate a **baseline ETL Job** on [SIGIR17](https://sites.google.com/site/limkwanhui/datacode) Attractions Dataset.

**Widget setup for passing parameters**

We'd be using **Azure Data Factory** to pass parameters into our Notebook, such as data source paths etc.

In [3]:
# Initialize to default values
dbutils.widgets.text("SRC_PATH", "/mnt/spark-poc/bronze/")
dbutils.widgets.text("POI_PATH", "data-sigir17/poiList-sigir17")
dbutils.widgets.text("VISITS_PATH", "data-sigir17/userVisits-sigir17")

# Initialize and Print variables
print ("Param -\'SRC_PATH':")
SRC_PATH = dbutils.widgets.get("SRC_PATH")
print (SRC_PATH)

print ("Param -\'POI_PATH':")
POI_PATH = dbutils.widgets.get("POI_PATH")
print (POI_PATH)

print ("Param -\'VISITS_PATH':")
VISITS_PATH = dbutils.widgets.get("VISITS_PATH")
print (VISITS_PATH)

Param -'SRC_PATH':
/mnt/spark-poc/bronze/
Param -'POI_PATH':
data-sigir17/poiList-sigir17
Param -'VISITS_PATH':
data-sigir17/userVisits-sigir17

-sandbox
# End-to-end Pipeline

<img src="https://i.imgur.com/Gf79JAp.png" style="width:1280px"/> <br>

#### Step 1: `Extract` Master Data: Points of Interest (POI) data from **Parsed**/**Raw** Zone
Our datasets resides in CSV format on ADLS Gen2, mimicking what would be the **Parsed Zone** in reality.

#### Step 2: `Extract` Transactional Data: User Vists (vists) data from **Parsed**/**Raw** Zone
Similar to the POI dataset - but the volume is much larger.

#### Step 3: `Transform` Data: Hash and aggregate columns (Feature Engineering) for Training Dataset
We use Spark functions to perform Data Transformations.

#### Step 4: `Train` Model: Alternating Least Squares (ALS) with Spark ML
A simple ALS model using Spark ML Library.

#### Step 5: `Serve` Model: Display User recommendations from model
We simply output the recommendation results of the model.

-sandbox
## Step 1: `Extract` Master Data: Points of Interest (POI) data from **Parsed**/**Raw** Zone
<br>
<img src="https://i.imgur.com/taWCf7l.png" style="width:1280px"/> <br>

In [6]:
poi_df = spark.read.csv(SRC_PATH + POI_PATH, header=True, inferSchema=True, sep=';')

In [7]:
display(poi_df)

poiID,poiName,lat,long,rideDuration,theme,theme2,theme3,theme4
1,Gadget's Go Coaster,33.810259,-117.918438,1.0,Kiddie,Roller Coaster,null,null
2,Astro Orbitor,28.418532,-81.579153,1.5,Spinning Ride,null,null,null
3,Mad Tea Party,33.813458,-117.918289,1.5,Family,Spinning Ride,null,null
4,Dumbo the Flying Elephant,33.81368,-117.918928,1.67,Family,Spinning Ride,null,null
5,Mr. Toad's Wild Ride,33.813311,-117.918697,2.0,Dark,Ride,null,Indoor
6,Snow White's Scary Adventures,33.813159,-117.919066,2.0,Ride,Family,Dark,Indoor
7,King Arthur Carrousel,33.813375,-117.918953,3.0,Family,Spinning Ride,null,null
8,Peter Pan's Flight,33.813087,-117.918704,3.0,Ride,Family,Dark,Indoor
9,Pinocchio's Daring Journey,33.813281,-117.919165,3.0,Family,Ride,Dark,Indoor
10,Alice in Wonderland,33.813168,-117.919387,4.0,Family,Ride,Dark,Indoor


In [8]:
display(poi_df.describe())

summary,poiID,poiName,lat,long,rideDuration,theme,theme2,theme3,theme4
count,113,113,113,113,113,113,91,29,60
mean,12.769911504424778,null,31.028864230088498,-99.26086128318586,10.410796460176991,null,null,null,null
stddev,7.932763040478664,null,2.7202062101323614,18.250540924833174,8.722473144289095,null,null,null,null
min,1,Alice in Wonderland,28.354907,-117.923284,1.0,Dark,3D Ride,3D Ride,Indoor
max,31,Walt Disney: One Man's Dream,33.815176,-81.545974,45.0,Water,Spinning Ride,Ride,Indoor


In [9]:
poi_df.createOrReplaceTempView('points')

-sandbox
## Step 2: `Extract` Transactional Data: User Vists (vists) data from **Parsed**/**Raw** Zone
<br>
<img src="https://i.imgur.com/7Y6LBDe.png" style="width:1280px"/> <br>

In [11]:
visits_df = spark.read.csv(SRC_PATH + VISITS_PATH, header=True, inferSchema=True, sep=';')

In [12]:
display(visits_df)

id,nsid,takenUnix,poiID,poiTheme,poiFreq,rideDuration,seqID
4618507320,10005149@N03,1273189220,21,Water,2497,450.0,1
4618512348,10005149@N03,1273196253,19,Roller Coaster,1629,420.0,1
4618601454,10005149@N03,1273281442,2,Spinning Ride,1754,90.0,2
4618030355,10005149@N03,1273291600,4,Family,2720,900.0,2
24469145379,100063617@N06,1450997221,27,Family,15118,1200.0,3
8221229213,10008488@N07,1353803001,10,Ride,2100,420.0,4
8222221938,10008488@N07,1353805071,14,Family,5171,120.0,4
8221228685,10008488@N07,1353805944,9,Ride,2906,240.0,4
8222250944,10008488@N07,1353816367,27,Family,15118,1200.0,4
8222238718,10008488@N07,1353816609,27,Family,15118,1200.0,4


In [13]:
display(visits_df.describe())

summary,id,nsid,takenUnix,poiID,poiTheme,poiFreq,rideDuration,seqID
count,332091,332091,332091,332091,332091,332091,332091,332091
mean,8.916292302139416E9,null,1.3233824075555675E9,15.975127299445031,null,6181.338365086678,740.7857015095262,4288.19415762547
stddev,6.226917245549216E9,null,7.424485813151902E7,8.695388902420218,null,5199.41535123863,488.5329445328107,3093.323953206581
min,102530213,10000151@N02,1187918299,1,Dark,162,60.0,1
max,29475731115,99987318@N03,1471870895,31,Water,18710,2700.0,11758


In [14]:
visits_df.createOrReplaceTempView('visits')

-sandbox
## Step 3: `Transform` Data: Hash and aggregate columns (Feature Engineering) for Training Dataset
<br>
<img src="https://i.imgur.com/mJOewKR.png" style="width:1280px"/> <br>

In [16]:
%sql
SELECT hash(nsid) AS user_hash_id, poiID, count(*) AS pictures_taken 
FROM visits GROUP BY 1,2

user_hash_id,poiID,pictures_taken
-1174038902,20,2
-1064654977,26,8
-600995796,8,9
-600995796,21,3
1791243274,16,1
1726597579,19,2
440723359,12,1
-1208326320,16,5
-2091637500,14,3
1241091588,23,1


In [17]:
train_df = spark.sql('''SELECT hash(nsid) AS user_hash_id, poiID, count(*) AS pictures_taken 
                        FROM visits GROUP BY 1,2''')

In [18]:
train_df.count()

Out[11]: 43271

-sandbox
## Step 4: `Train` Model: Alternating Least Squares (ALS) with Spark ML
<br>
<img src="https://i.imgur.com/4dg6JWg.png" style="width:1280px"/> <br>

In [20]:
from pyspark.ml.recommendation import ALS

recommender = ALS(userCol="user_hash_id", itemCol="poiID", ratingCol="pictures_taken", coldStartStrategy="drop")

model = recommender.fit(train_df)

In [21]:
recommendations = model.recommendForAllUsers(10)

-sandbox
## Step 5: `Serve` Model: Display User recommendations from model
<br>
<img src="https://i.imgur.com/DZdUjW7.png" style="width:1280px"/> <br>

In [23]:
display(recommendations)

user_hash_id,recommendations
-258415999,"List(List(28, 0.9997095), List(29, 0.6728074), List(25, 0.56879205), List(26, 0.5686367), List(13, 0.53372055), List(16, 0.51956207), List(24, 0.36034498), List(31, 0.29312992), List(22, 0.26266843), List(1, 0.22873905))"
-1606259,"List(List(31, 3.8260562), List(28, 3.1715412), List(16, 1.9964302), List(13, 1.5200495), List(9, 1.4528027), List(26, 1.4474192), List(8, 1.3769071), List(24, 1.366722), List(25, 1.3446751), List(1, 1.3213458))"
1141394441,"List(List(28, 19.272276), List(13, 10.470584), List(8, 8.870338), List(1, 7.666995), List(24, 5.9851913), List(26, 5.9294324), List(29, 5.362789), List(22, 3.957798), List(19, 3.8376896), List(23, 3.5431447))"
1198433351,"List(List(29, 14.505946), List(13, 13.344306), List(25, 11.78423), List(28, 9.942785), List(1, 9.767961), List(16, 8.46022), List(24, 7.9596925), List(18, 5.249641), List(14, 3.2846732), List(21, 3.0475452))"
1347840121,"List(List(29, 36.1865), List(25, 32.263836), List(13, 23.207312), List(27, 20.687887), List(16, 16.445297), List(1, 13.97473), List(24, 12.507899), List(31, 9.040902), List(21, 6.268028), List(2, 5.6851554))"
1660066971,"List(List(18, 2.7616663), List(26, 1.4765413), List(28, 1.2958109), List(20, 1.0503119), List(17, 0.99235135), List(19, 0.9890706), List(1, 0.90295935), List(14, 0.796902), List(6, 0.7925916), List(27, 0.68369555))"
-1552308127,"List(List(26, 1.2199008), List(20, 1.1706417), List(18, 1.0002537), List(19, 0.98777246), List(28, 0.80696756), List(1, 0.73977584), List(6, 0.7176664), List(14, 0.54800844), List(15, 0.4672935), List(30, 0.45237938))"
-1354247967,"List(List(22, 16.14043), List(12, 10.037353), List(29, 6.9712415), List(16, 5.8420596), List(17, 5.6827097), List(7, 5.5806193), List(30, 4.9236407), List(25, 4.8566546), List(27, 4.0816145), List(8, 3.3539143))"
170223113,"List(List(26, 1.2089937), List(18, 1.1964656), List(20, 1.1841354), List(19, 0.99400127), List(28, 0.8099081), List(1, 0.74277437), List(6, 0.7292733), List(14, 0.5697524), List(15, 0.46887064), List(30, 0.44901314))"
842187073,"List(List(18, 7.6120343), List(1, 3.9661143), List(20, 3.8082988), List(26, 3.405976), List(28, 3.3142228), List(14, 2.9704504), List(31, 2.8517911), List(6, 2.8240514), List(19, 2.8065815), List(15, 2.426722))"
